# DDSPの試験実装
### 構成
0. ライブラリのインストール
1. データのダウンロードと配置
2. 前処理
3. DDSPDeocderモジュール
4. MulchScaleloss
5. 実行

### 使用時の流れ
1. データローダーを用いてgoogle driveにデータセットをロードする（初回のみ）
2. 各wavファイルに対しあらかじめf0, loudnessを計算しgoogle driveに.ptファイルとして保存する
3. 計算を実行する

### 本モデルの機能
1. 任意の楽器の単旋律からf0(t), loudness(t)を抽出（当初librosaだったが、GPUを使って時短するという観点でtorchcrepeを用いる）
※今回はz(t)を入力としていない関係で、入力音声の時系列的変化（すなわちトランペットの吹き方の時系列変化）をとらえられない。声を入力とした場合、歌詞の情報は完全に排除された出力が得られる構成になっている

2. 抽出した情報からGRUを通して各時刻tに対し128次元の隠れ層状態を取得する。（最終出力は無視するため、_で受け取る）

3. 隠れ状態の情報を線形変換して倍音の構成(B,T,60)、全体音量(B,T,1)、ノイズ(B,T,ノイズの周波数バンド構成)を推定する




## 0. ライブラリのインストール

In [ ]:
# 必要なライブラリのインストール
!pip install torchcrepe librosa numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.7 MB/s eta 0:00:00


## パラメータ設定

In [2]:
# ==========================================
# 0. 設定 (Hyperparameters)
# ==========================================
SAMPLE_RATE = 16000
N_FFT = 2048
HOP_LENGTH = 64  # 時間分解能を高めるため小さめに設定 (4ms)
BLOCK_SIZE = 512 # 1回の学習で扱うフレーム数

# シンセサイザー設定
N_HARMONICS = 60 # 倍音の数
N_NOISE_BANDS = 65 # ノイズフィルタの周波数バンド数

# モデル設定
HIDDEN_SIZE = 256
N_LAYERS = 1

# --- 設定 ---
OUTPUT_DIR = "data/features"

## 1. データのダウンロードとリバーブの除去



In [3]:

# 1. 必要なライブラリのインストール
!pip install -U demucs soundfile librosa

import torch
import torchaudio
import librosa
import soundfile as sf
import numpy as np
from demucs.pretrained import get_model
from demucs.apply import apply_model

def separate_with_demucs():
    print("1. モデルをロード中 (htdemucs)...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Demucsの学習済みモデルを取得
    model = get_model('htdemucs').to(device)
    model.eval()

    print("2. 音源を準備中...")
    # 元音源を取得
    filename = librosa.ex('trumpet')
    # Demucsは 44.1kHz で動作するため、ここでリサンプリングして読み込む
    wav, sr = librosa.load(filename, sr=44100, mono=False)

    # モノラルの場合、ステレオ(2ch)に拡張する（モデルの仕様）
    if wav.ndim == 1:
        wav = np.stack([wav, wav]) # [2, T]

    # Tensor化: [Batch, Channels, Time]
    wav_tensor = torch.tensor(wav).unsqueeze(0).to(device)

    print("3. 分離処理を実行中...")
    # モデル適用 (shifts=1 は高速化のため。画質ならshifts=4推奨)
    with torch.no_grad():
        sources = apply_model(model, wav_tensor, shifts=1, split=True, overlap=0.25)

    # sourcesの形状: [Batch, Source_Count, Channels, Time]
    # htdemucsのソース順序: ["drums", "bass", "other", "vocals"]
    # トランペットのような主旋律は "vocals" に入ります
    vocals_idx = model.sources.index('vocals')
    vocals_wav = sources[0, vocals_idx].cpu().numpy() # [Channels, Time]

    # ステレオをモノラルに戻す（平均）
    vocals_mono = vocals_wav.mean(axis=0)

    print("4. 保存中...")
    # 学習用の 16kHz にリサンプリングして保存
    # (soundfileなどを使って手動リサンプリングするのは面倒なので、librosaで再ロード時にやってもいいですが
    #  ここでは簡易的なリサンプリングをして保存します)

    output_path = "dry_trumpet.wav"

    # 44.1k -> 16k へリサンプリング (librosaを使用)
    vocals_resampled = librosa.resample(vocals_mono, orig_sr=44100, target_sr=16000)

    # 保存 (soundfileを使用＝torchaudioのエラー回避)
    sf.write(output_path, vocals_resampled, 16000)

    print(f"成功！ '{output_path}' を作成しました。")

    # 確認再生
    from IPython.display import Audio, display
    display(Audio(output_path, rate=16000))

if __name__ == "__main__":
    separate_with_demucs()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.4 MB/s eta 0:00:00
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.whl size=78388 sha256=9186ece224a1c5246625383753ba1574e2b4a853d5b7ae109f0001afc63cc366
  Stored in directory: /root/.cache/pip/wheels/1b/0c/20/a3b3daa1f9b65c8b0445729f94740ec335d0f86f1066c5c414
  Created wheel for julius: filename=julius-0.2.7-

100%|██████████| 80.2M/80.2M [00:01<00:00, 66.3MB/s]


2. 音源を準備中...
3. 分離処理を実行中...
4. 保存中...
成功！ 'dry_trumpet.wav' を作成しました。


## 前処理

In [6]:
import os
import torch
import torchcrepe
import librosa
import numpy as np
import soundfile as sf


# 保存先作成
os.makedirs(OUTPUT_DIR, exist_ok=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def preprocess_librosa_sample():
    print("librosa公式のサンプル音源(Trumpet)を取得中...")

    # 1. librosa内蔵のトランペット音源をロード
    # 初回は自動的にキャッシュへダウンロードされます
    try:
        # filename = librosa.ex('trumpet')
        filename = "dry_trumpet.wav"      # <-- さっき作ったファイルを使う
    except Exception as e:
        # 万が一ネットワークエラー等の場合
        print(f"ダウンロードエラー: {e}")
        return

    print(f"Loaded: {filename}")

    # 2. ロード & リサンプリング
    y, sr = librosa.load(filename, sr=SAMPLE_RATE)

    # 計算用に長さをHOP_LENGTHの倍数に切り捨て
    n_frames = len(y) // HOP_LENGTH
    y = y[:n_frames * HOP_LENGTH]

    # --- A. f0 (Pitch) Extraction with TorchCrepe ---
    print("Extracting Pitch (f0)...")
    audio_tensor = torch.tensor(y).unsqueeze(0).to(device)

    # トランペットは高音域も含むのでfmaxは2000でOK
    f0 = torchcrepe.predict(
        audio_tensor,
        SAMPLE_RATE,
        hop_length=HOP_LENGTH,
        fmin=50,
        fmax=2000,
        model='full',
        batch_size=1024,
        device=device,
        decoder=torchcrepe.decode.viterbi
    )
    f0 = f0.squeeze(0).cpu().numpy()

    # --- B. Loudness Extraction ---
    print("Extracting Loudness...")
    stft = librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH)
    magnitude = np.abs(stft)
    loudness = librosa.amplitude_to_db(magnitude.mean(axis=0), ref=np.max)
    loudness = loudness.astype(np.float32)

    # --- C. Save ---
    min_len = min(len(f0), len(loudness))
    f0 = f0[:min_len]
    loudness = loudness[:min_len]
    audio_len = min_len * HOP_LENGTH
    y = y[:audio_len]

    # 保存ファイル名は violin_training_data.pt のままでも、新しい dataset.py で読めますが
    # 混乱を避けるため trumpet_training_data.pt とし、後述の学習コードで読み込み先を変えます
    save_path = os.path.join(OUTPUT_DIR, "trumpet_training_data.pt")

    torch.save({
        'audio': torch.from_numpy(y),
        'f0': torch.from_numpy(f0),
        'loudness': torch.from_numpy(loudness)
    }, save_path)

    print(f"完了！ 保存先: {save_path}")

if __name__ == "__main__":
    preprocess_librosa_sample()

ModuleNotFoundError: No module named 'torchcrepe'

## 4. モデル定義(HarmonySynthesizer, Noise_synthesizer, DDSP,LossF)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

# ==========================================
# 2. Synthesizer Components
# ==========================================

class HarmonicSynthesizer(nn.Module):
    def __init__(self, sample_rate, hop_length):
        super().__init__()
        self.sample_rate = sample_rate
        self.hop_length = hop_length

    def forward(self, f0, harmonics_dist, global_amp):
        """
        f0: [B, T_frames, 1]
        harmonics_dist: [B, T_frames, N_harmonics] (和が1になる分布)
        global_amp: [B, T_frames, 1]
        """
        # 1. Upsampling (Frame -> Sample)
        # f0, amp, dist 全てを音声サンプルの長さに引き伸ばす
        f0_up = F.interpolate(f0.transpose(1, 2), scale_factor=self.hop_length, mode='linear').transpose(1, 2)
        amp_up = F.interpolate(global_amp.transpose(1, 2), scale_factor=self.hop_length, mode='linear').transpose(1, 2)
        harm_up = F.interpolate(harmonics_dist.transpose(1, 2), scale_factor=self.hop_length, mode='linear').transpose(1, 2)

        # 2. Phase Calculation (Anti-aliasing処理などは省略した簡易版)
        harmonic_indices = torch.arange(1, harmonics_dist.shape[-1] + 1, device=f0.device).float()

        # 全倍音の周波数: [B, T_samples, 1] * [1, 1, N_harm] -> [B, T_samples, N_harm]
        frequencies = f0_up * harmonic_indices.view(1, 1, -1)

        # 積分して位相を計算
        phases = 2 * np.pi * torch.cumsum(frequencies / self.sample_rate, dim=1) # phases = [B, T, N_harm]

        # 3. Waveform Generation
        # sin(phase) * magnitude
        harmonic_waves = torch.sin(phases) * harm_up # [B, T, N_harm] * [B, T, N_harm] = [B, T, N_harm]

        # 全倍音を合成
        waveform = torch.sum(harmonic_waves, dim=-1, keepdim=True)

        # 全体音量を適用
        return waveform * amp_up

class FilteredNoiseSynthesizer(nn.Module):
    def __init__(self, block_size, sample_rate):
        super().__init__()
        self.block_size = block_size # 周波数領域での処理単位

    def forward(self, filter_coeffs):
        """
        filter_coeffs: [B, T_frames, N_noise_bands] -> ノイズの周波数特性
        """
        B, T, N = filter_coeffs.shape
        # 時間領域でフィルタ係数をアップサンプリングするのは重いため、
        # 論文では「周波数領域での掛け算」として実装することが多いです。

        # 簡易実装:
        # 1. ホワイトノイズ生成 [B, T_samples]
        T_samples = T * HOP_LENGTH
        noise = torch.rand(B, T_samples, device=filter_coeffs.device) * 2 - 1

        # 2. フィルタ係数をアップサンプリングして適用
        # 本当はLTV(Linear Time Variant)フィルタですが、簡易的に振幅変調として実装します
        # (厳密なDDSPは周波数領域でWindowingをして畳み込みますが、ここでは直感的な実装にします)

        filter_up = F.interpolate(filter_coeffs.transpose(1, 2), scale_factor=HOP_LENGTH, mode='linear').transpose(1, 2)
        # フィルタ係数が周波数バンドごとのゲインを表すと仮定 (簡易化)
        # 本来はここからインパルス応答を作り、畳み込みます。

        # 今回は「学習可能なノイズエンベロープ」として、ノイズ成分全体を1つのバンドとみなして合成します
        # ※厳密な実装にはFFT畳み込みが必要です

        noise_gain = torch.mean(filter_up, dim=-1, keepdim=True) # 平均ゲイン
        return noise.unsqueeze(-1) * noise_gain


# ==========================================
# 3. The DDSP Model (Decoder)
# ==========================================

class DDSP(nn.Module):
    def __init__(self):
        super().__init__()
        # Input: f0(1) + loudness(1) = 2
        self.mlp = nn.Sequential(
            nn.Linear(2, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )
        self.gru = nn.GRU(256, HIDDEN_SIZE, num_layers=N_LAYERS, batch_first=True)

        # Parameters Output
        self.dense_amp = nn.Linear(HIDDEN_SIZE, 1)        # Overall Amplitude
        self.dense_harm = nn.Linear(HIDDEN_SIZE, N_HARMONICS) # Harmonic Distribution
        self.dense_noise = nn.Linear(HIDDEN_SIZE, N_NOISE_BANDS) # Noise Filter

        # Synths
        self.harmonic_synth = HarmonicSynthesizer(SAMPLE_RATE, HOP_LENGTH)
        self.noise_synth = FilteredNoiseSynthesizer(BLOCK_SIZE, SAMPLE_RATE)

    def forward(self, f0_hz, f0_norm, loudness_norm):
        # 1. Preprocessing (MLP)
        # f0は値が大きいので対数化したり、Embeddingしたりするのが一般的ですが
        # ここではRaw値を使います (Normalizationは済んでいる前提)
        x = torch.cat([f0_norm, loudness_norm], dim=-1)
        x = self.mlp(x)

        # 2. Recurrent Processing
        # hiddenは使いません（DDSPは全時刻の出力が必要）
        x, _ = self.gru(x)

        # 3. Parameter Projection
        # 論文通り、正の値にするため exp や sigmoid, softmax を使います

        # A. Harmonic Params
        amp = torch.exp(self.dense_amp(x)) # 振幅は必ず正 [B, T, 1]
        harm_dist = F.softmax(self.dense_harm(x), dim=-1) # 分布は合計1 [B, T, 60]

        # B. Noise Params
        noise_coeffs = torch.exp(self.dense_noise(x)) # [B, T, 65]

        # 4. Synthesis
        audio_harm = self.harmonic_synth(f0_hz, harm_dist, amp)
        audio_noise = self.noise_synth(noise_coeffs)

        # 5. Combine
        final_audio = audio_harm + audio_noise
        return final_audio.squeeze(-1)

# ==========================================
# 4. Loss Function (Multi-Scale Spectral Loss)
# ==========================================

def multiscale_spectral_loss(pred, target, fft_sizes=[2048, 1024, 512, 256, 128, 64]):
    loss = 0.0
    for n_fft in fft_sizes:
        hop = n_fft // 4

        # STFT
        pred_stft = torch.stft(pred, n_fft=n_fft, hop_length=hop, window=torch.hann_window(n_fft).to(pred.device), return_complex=True)
        target_stft = torch.stft(target, n_fft=n_fft, hop_length=hop, window=torch.hann_window(n_fft).to(target.device), return_complex=True)

        pred_mag = torch.abs(pred_stft)
        target_mag = torch.abs(target_stft)

        # Linear Loss (L1)
        loss += F.l1_loss(pred_mag, target_mag)

        # Log Loss (L1)
        loss += F.l1_loss(torch.log(pred_mag + 1e-7), torch.log(target_mag + 1e-7))

    return loss

## 3. 学習の実行

In [8]:

#    (以下は学習ループのメイン部分のみ記述します)

# ==========================================
# データセットクラス (バイオリン1曲用)
# ==========================================
class SingleViolinDataset(Dataset):
    def __init__(self, feature_path, crop_len_sec=4.0, sample_rate=16000, hop_length=64):
        self.data = torch.load(feature_path)
        self.crop_len_sec = crop_len_sec
        self.sample_rate = sample_rate
        self.hop_length = hop_length
        self.n_frames_per_sample = int(crop_len_sec * sample_rate / hop_length)

        # データの総フレーム数
        self.total_frames = len(self.data['f0'])

        # f0の範囲
        self.f_min = 0.0
        self.f_max = 2000.0  # crepeのfmaxと同じにする

        # loudnessの範囲 (dB) (例: -120dB ~ 0dB)
        # librosa.amplitude_to_db(ref=np.max) なので最大値は0です
        self.l_min = -120.0 # 無音付近の下限
        self.l_max = 0.0

    def __len__(self):
        # 1つの長いファイルからランダムサンプリングするので、
        # epochあたりのステップ数を適当に大きく設定します
        return 1000

    def __getitem__(self, idx):
        # ランダムな位置から切り出す
        start_frame = np.random.randint(0, self.total_frames - self.n_frames_per_sample)
        end_frame = start_frame + self.n_frames_per_sample

        f0 = self.data['f0'][start_frame:end_frame]
        loudness = self.data['loudness'][start_frame:end_frame]

        start_sample = start_frame * self.hop_length
        end_sample = end_frame * self.hop_length
        audio = self.data['audio'][start_sample:end_sample]

        # 1. f0 の正規化: [0, 2000] -> [0.0, 1.0]
        # (対数正規化の方がより良いですが、まずは線形で十分です)
        f0_norm = (f0 - self.f_min) / (self.f_max - self.f_min + 1e-7)

        # 2. Loudness の正規化: [-120, 0] -> [0.0, 1.0]
        # 値が小さいほど無音(0.0)、0dBに近いほど大音量(1.0)になるようにします
        loudness_norm = (loudness - self.l_min) / (self.l_max - self.l_min + 1e-7)
        loudness_norm = torch.clamp(loudness_norm, 0.0, 1.0) # 範囲外をクリップ

        return {
            'f0': f0.unsqueeze(-1),             # Synth用: 元のHz単位 [T, 1]
            'f0_norm': f0_norm.unsqueeze(-1),   # MLP入力用: 正規化済み [T, 1]
            'loudness_norm': loudness_norm.unsqueeze(-1), # MLP入力用: 正規化済み [T, 1]
            'audio': audio
        }

# ==========================================
# 学習実行
# ==========================================
def train_violin():
    # 1. データセットとローダー
    # trumpet_training_data.pt がない場合は violin_training_data.pt に戻してください
    dataset = SingleViolinDataset("data/features/trumpet_training_data.pt")
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    # 2. モデルとオプティマイザ
    model = DDSP().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # スケジューラの定義
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=5,
    )

    print("学習開始...")

    # 3. ループ
    for epoch in range(1, 101):
        total_loss = 0

        # --- 内側のループ（バッチごとの処理） ---
        for i, batch in enumerate(dataloader):
            f0_hz = batch['f0'].to(device)
            f0_norm = batch['f0_norm'].to(device)
            loud_norm = batch['loudness_norm'].to(device)
            target_audio = batch['audio'].to(device)

            optimizer.zero_grad()
            generated_audio = model(f0_hz, f0_norm, loud_norm)
            loss = multiscale_spectral_loss(generated_audio, target_audio)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # 途中経過の表示（ここはループの中）
            if i % 10 == 0:
                 # ここでは batchごとの loss を表示するだけ
                 print(f"  Step {i} | Loss: {loss.item():.4f}")

        # --- 【エポック終了後の処理】 ---

        # 1エポック分の平均ロスを計算
        avg_loss = total_loss / len(dataloader)

        # スケジューラに平均ロスを渡す
        # (エラーの原因は、この行が上のループの中にあったことでした)
        scheduler.step(avg_loss)

        print(f"=== Epoch {epoch} Finished | Avg Loss: {avg_loss:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f} ===")

        # 定期保存
        if epoch % 10 == 0:
            torch.save(model.state_dict(), f"ddsp_trumpet_epoch{epoch}.pth")

## `SingleViolinDataset` クラスの詳細

このクラスは、DDSPモデルの学習に使用するデータセットを準備するためのカスタム`Dataset`クラスです。
特定の音源（今回はトランペット）から抽出されたf0（ピッチ）とloudness（音量）の時系列データ、および元の音声波形を扱います。

### 主要な機能

1.  **データのロード**: 初期化時に、前処理済みデータが保存されている`.pt`ファイル（例: `trumpet_training_data.pt`）を読み込みます。
2.  **クロップ（切り出し）**: 学習効率のため、長い音源データから指定された秒数（`crop_len_sec`、デフォルトは4秒）の短いセグメントをランダムに切り出して使用します。
3.  **特徴量の正規化**:
    *   **f0 (ピッチ)**: f0の値はHz単位であり、モデルへの入力に適した範囲にするために0.0から1.0の間に線形正規化されます。元のHz単位のf0もシンセサイザーで使用するために保持されます。
    *   **Loudness (音量)**: LoudnessはdB単位で表現されており、こちらも0.0から1.0の範囲に正規化されます。0dB（最大音量）が1.0、無音に近い-120dBが0.0にマッピングされます。
4.  **データ提供**: `__getitem__`メソッドが呼び出されると、ランダムに選択されたデータセグメント（f0、正規化f0、正規化loudness、元の音声波形）がPyTorchのテンソルとして返されます。

### クラス内の重要な変数

*   `feature_path`: 前処理済みの特徴量ファイル（`.pt`）へのパス。
*   `crop_len_sec`: 各トレーニングサンプルで切り出す音声の秒数。
*   `sample_rate`: 音声のサンプリングレート（デフォルト16000Hz）。
*   `hop_length`: 特徴量抽出時のホップサイズ（デフォルト64フレーム）。
*   `n_frames_per_sample`: `crop_len_sec`に対応するフレーム数。
*   `f_min`, `f_max`: f0の正規化に使用される最小値と最大値。
*   `l_min`, `l_max`: loudnessの正規化に使用される最小値と最大値。

このデータセットクラスは、大量の時系列音声データからモデルが効率的に学習できるように、動的にデータを準備する役割を担っています。

## 4. 実行

In [ ]:
# --- メイン実行 ---
if __name__ == "__main__":
    # 前回のクラス定義やLoss関数がメモリ上にある状態で実行してください
    train_violin()

## 5. 音の確認用コード

In [1]:
import torch
import IPython.display as ipd
import glob
import os

# 1. 保存された重みファイルを探す
list_of_files = glob.glob('ddsp_trumpet_*.pth')
if len(list_of_files) == 0:
    print("エラー: 学習済みの .pth ファイルが見つかりません。")
else:
    # 最新のファイルを選択
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"Loading model weights from: {latest_file}")

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 2. モデル再構築
    model = DDSP().to(device)
    model.load_state_dict(torch.load(latest_file, map_location=device))
    model.eval()

    # 3. 音声生成
    with torch.no_grad():
        # データセットが未定義なら作成 (dry_trumpet.wav または trumpet_training_data.pt)
        if 'dataset' not in globals():
             # ファイル名は実際に作成した .pt ファイルに合わせてください
            dataset = SingleViolinDataset("data/features/trumpet_training_data.pt")

        sample = dataset[0]

        # --- 修正ポイント ---
        # Datasetクラスですでに正規化された値('f0_norm', 'loudness_norm')が作られているので、
        # それを直接利用します。再計算は不要です。

        # 1. シンセサイザ用のHz単位F0
        f0_hz = sample['f0'].unsqueeze(0).to(device)

        # 2. ニューラルネット入力用の正規化済みデータ
        f0_norm = sample['f0_norm'].unsqueeze(0).to(device)
        loudness_norm = sample['loudness_norm'].unsqueeze(0).to(device)

        # 3. モデル実行
        # 引数: (物理量F0, 正規化F0, 正規化Loudness)
        audio_out = model(f0_hz, f0_norm, loudness_norm)

        # --- 再生 ---
        print("\n=== 生成された音声 (Synthesized) ===")
        ipd.display(ipd.Audio(audio_out.cpu().numpy().squeeze(), rate=16000))

        print("\n=== 元の音声 (Original) ===")
        ipd.display(ipd.Audio(sample['audio'].numpy(), rate=16000))

エラー: 学習済みの .pth ファイルが見つかりません。


In [ ]:
# 推論時のデバッグ用コード例
# モデルのforwardを少し改造して、harmとnoiseを別々に返すモードを作るか、
# あるいは以下のように手動で内部の関数を呼ぶと確実です。

# (推論セルの中で)
with torch.no_grad():
    # 1. MLPを通す
    x = torch.cat([f0_norm, loudness_norm], dim=-1)
    x = model.mlp(x)
    x, _ = model.gru(x)

    # 2. パラメータ計算
    amp = torch.exp(model.dense_amp(x))
    harm_dist = F.softmax(model.dense_harm(x), dim=-1)
    noise_coeffs = torch.exp(model.dense_noise(x))

    # 3. 別々に合成
    audio_harm = model.harmonic_synth(f0, harm_dist, amp).squeeze()
    audio_noise = model.noise_synth(noise_coeffs).squeeze()

    # 合成
    audio_full = audio_harm + audio_noise

# 4. 保存して聴き比べ
ipd.display(ipd.Audio(audio_harm.cpu().numpy().squeeze(), rate=16000))
ipd.display(ipd.Audio(audio_noise.cpu().numpy().squeeze(), rate=16000))
ipd.display(ipd.Audio(audio_full.cpu().numpy().squeeze(), rate=16000))